In [11]:
import os
for file in os.listdir(path='.'):
    if file.endswith('.html'):
        with open(file) as fp:
            html = fp.read()
    break

In [12]:
print(html)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8" />
<title>Sumner Crakehall - A Wiki of Ice and Fire</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Sumner_Crakehall","wgTitle":"Sumner Crakehall","wgCurRevisionId":238410,"wgRevisionId":238410,"wgArticleId":4899,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Characters without images","House Crakehall","Characters from the Westerlands","Lords of Crakehall","Nobles"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Janu